In [1]:
import numpy as np 
import pandas as pd
from contextlib import redirect_stdout
from proj_logging import logger

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support ,precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
import tensorflow as tf
import os
from proj_logging import logger
from dataset_handling import get_config_metadata, write_config_metadata
from pyspark import SparkContext
from pyspark.sql import SparkSession, Row
from pyspark import SparkConf
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import CountVectorizer
from timeit import default_timer as timer
from pyspark.ml.stat import Summarizer
from pyspark.sql.functions import udf, split, monotonically_increasing_id, explode, col, mean, concat, lit
from pyspark.sql.types import DoubleType, ArrayType, StringType, MapType, StructType, StructField
import warnings
import pandas as pd
import numpy as np
from dataset_handling import book_train_test_split
import torch
from feature_engineering import extract_features

2024-11-25 16:41:11.764863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732552871.784043  241008 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732552871.790068  241008 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 16:41:11.812708: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [43]:
e = pd.read_parquet("data/primary_authors/document_embeddings.parquet")
e

,features,is_train,author_id
0,"[0.28832898, 0.38870975, 0.13632962, 0.520275,...",False,10294
1,"[0.4572123, 0.3324129, 0.24623187, 0.5438165, ...",False,10294
2,"[0.46877238, 0.30636883, 0.30725992, 0.4327013...",False,10294
3,"[0.3527186, 0.34149382, 0.30431616, 0.47463518...",False,10294
4,"[0.41601232, 0.5375397, 0.2180003, 0.43813646,...",False,10294
...,...,...,...
19004,"[0.37158808, 0.43231353, 0.27672166, 0.5070651...",False,5036
19005,"[0.27637783, 0.32261384, 0.27342793, 0.3701268...",False,5036
19006,"[0.52867526, 0.2188764, 0.4117935, 0.47598585,...",False,5036
19007,"[0.39144635, 0.38395822, 0.40404236, 0.5401964...",False,5036


In [3]:
from pyspark.ml.functions import array_to_vector
from pyspark.mllib.evaluation import MulticlassMetrics

# dataset = spark.read.parquet('data/primary_authors/document_embeddings.parquet')
dataset = spark.read.parquet('data/all/document_embeddings.parquet')

In [4]:
dataset.take(1)

[Row(book_id=12, final_vector=DenseVector([0.0091, 0.0575, -0.1319, -0.0732, 0.1199, 0.0529, 0.0602, -0.0561, -0.047, 1.3077, -0.1412, 0.0283, 0.0318, -0.0996, 0.0212, -0.0221, -0.1098, 0.9808, -0.1067, -0.0158, 0.0431, -0.0005, -0.0805, 0.0001, 0.0622, -0.0265, -0.0877, -0.0697, 0.0538, 0.0808, -0.0748, 0.0217, 0.0193, 0.0942, 0.069, -0.0642, 0.0748, 0.0614, -0.1318, -0.0138, 0.0468, 0.0264, 0.0032, -0.0297, 0.0206, 0.02, 0.0238, 0.0321, 0.0185, 0.0126, -0.0216, 0.0734, 0.0485, 0.022, 0.0053, -0.0116, -0.0443, -0.0204, -0.0251, -0.0702, -0.0263, 0.038, -0.01, 0.0958, -0.0039, 0.0184, 0.0301, 0.0009, 0.0106, 0.0358, -0.0265, 0.098, 0.0177, -0.0239, 0.1004, 0.0262, -0.0588, 0.0549, -0.0511, 0.0094, 0.0264, 0.1201, -0.087, 0.1064, 0.0449, -0.0766, 0.367, -0.197, 0.0181, -0.0099, -0.1296, -0.0379, 0.0033, -0.0438, 0.0849, 0.096, 0.0608, -0.0338, 0.0109, -0.0044, 0.0206, -0.017, -0.0296, 0.0028, 0.0158, -0.8623, 0.1151, 0.0587, 0.0012, -0.0064, -0.0452, -0.1528, 0.0304, -0.0347, -0.022, 0.

In [35]:
path = 'data/primary_authors/dataset.parquet'
path[:path.rindex('/')+1]

'data/primary_authors/'

In [31]:
dataset.select("author_id", "label").distinct().write.csv(

[Row(author_id='5036', label=0.0),
 Row(author_id='3258', label=1.0),
 Row(author_id='10294', label=2.0),
 Row(author_id='3304', label=3.0)]

In [69]:
t = MulticlassMetrics(dataset.select("prediction", "label").rdd)


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `prediction` cannot be resolved. Did you mean one of the following? [`features`, `is_train`, `author_id`].;
'Project ['prediction, 'label]
+- Relation [features#18,is_train#19,author_id#20] parquet


In [39]:
t.confusionMatrix().toArray().tofile(

array([[7310.,  610.,    0.,  183.],
       [1166., 2800.,    0.,    0.],
       [2205.,  118.,    0.,   59.],
       [ 567.,   39.,    0.,  156.]])

In [34]:
embeddings_index = {}

# Lead the txt into mem
with open("glove.840B.300d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.strip().split(' ')
        word = values[0]
        try:
            embedding = [float(x) for x in values[1:]]
            embeddings_index[word] = embedding
            print(embedding)
            break
        except ValueError:
            continue

[-0.082752, 0.67204, -0.14987, -0.064983, 0.056491, 0.40228, 0.0027747, -0.3311, -0.30691, 2.0817, 0.031819, 0.013643, 0.30265, 0.0071297, -0.5819, -0.2774, -0.062254, 1.1451, -0.24232, 0.1235, -0.12243, 0.33152, -0.006162, -0.30541, -0.13057, -0.054601, 0.037083, -0.070552, 0.5893, -0.30385, 0.2898, -0.14653, -0.27052, 0.37161, 0.32031, -0.29125, 0.0052483, -0.13212, -0.052736, 0.087349, -0.26668, -0.16897, 0.015162, -0.0083746, -0.14871, 0.23413, -0.20719, -0.091386, 0.40075, -0.17223, 0.18145, 0.37586, -0.28682, 0.37289, -0.16185, 0.18008, 0.3032, -0.13216, 0.18352, 0.095759, 0.094916, 0.008289, 0.11761, 0.34046, 0.03677, -0.29077, 0.058303, -0.027814, 0.082941, 0.1862, -0.031494, 0.27985, -0.074412, -0.13762, -0.21866, 0.18138, 0.040855, -0.113, 0.24107, 0.3657, -0.27525, -0.05684, 0.34872, 0.011884, 0.14517, -0.71395, 0.48497, 0.14807, 0.62287, 0.20599, 0.58379, -0.13438, 0.40207, 0.18311, 0.28021, -0.42349, -0.25626, 0.17715, -0.54095, 0.16596, -0.036058, 0.08499, -0.64989, 0.075

In [31]:
dataset.take(1)

[Row(features=[-0.033117614686489105, 0.12483888119459152, -0.1549590528011322, -0.05444326251745224, 0.10045762360095978, -0.05532223358750343, -0.0026100422255694866, -0.08408316224813461, -0.03065658174455166, 2.3531155586242676, -0.11757563799619675, -0.0012547832448035479, 0.06489349901676178, -0.07668668776750565, -0.14405636489391327, -0.07120014727115631, -0.08332885056734085, 0.932112455368042, -0.16846469044685364, -0.04060450568795204, 0.02029799297451973, -0.07106799632310867, -0.06717848777770996, -0.00455648684874177, 0.0029725641943514347, 0.025969045236706734, -0.07220955193042755, -0.05050615593791008, 0.03428267315030098, -0.07148827612400055, -0.05324846878647804, 0.1629950851202011, -0.0616370253264904, 0.06018863990902901, 0.0775580033659935, -0.0882159098982811, -0.03288496285676956, 0.0039817257784307, -0.07003581523895264, -0.09210877865552902, -0.009599561803042889, 0.11958678811788559, -0.012092802673578262, -0.0970168188214302, 0.08391684293746948, 0.00256107

In [25]:
dataset.select("author_id", array_to_vector("features").alias("features"), "is_train")

DataFrame[author_id: int, features: vector, is_train: boolean]

In [26]:
dataset.take(1)

[Row(features=[-0.033117614686489105, 0.12483888119459152, -0.1549590528011322, -0.05444326251745224, 0.10045762360095978, -0.05532223358750343, -0.0026100422255694866, -0.08408316224813461, -0.03065658174455166, 2.3531155586242676, -0.11757563799619675, -0.0012547832448035479, 0.06489349901676178, -0.07668668776750565, -0.14405636489391327, -0.07120014727115631, -0.08332885056734085, 0.932112455368042, -0.16846469044685364, -0.04060450568795204, 0.02029799297451973, -0.07106799632310867, -0.06717848777770996, -0.00455648684874177, 0.0029725641943514347, 0.025969045236706734, -0.07220955193042755, -0.05050615593791008, 0.03428267315030098, -0.07148827612400055, -0.05324846878647804, 0.1629950851202011, -0.0616370253264904, 0.06018863990902901, 0.0775580033659935, -0.0882159098982811, -0.03288496285676956, 0.0039817257784307, -0.07003581523895264, -0.09210877865552902, -0.009599561803042889, 0.11958678811788559, -0.012092802673578262, -0.0970168188214302, 0.08391684293746948, 0.00256107

In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("SparkTFIDF") \
            .set('spark.local.dir', '/media/volume/team11data/tmp') \
            .set('spark.driver.memory', '50G') \
            .set('spark.driver.maxResultSize', '25G') \
            .set('spark.executor.memory', '10G')
sc = SparkContext(conf = conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)
# dataset = spark.read.parquet("data/primary_authors/dataset.parquet")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 16:41:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/25 16:41:23 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
dataset.take(1)

[Row(book_id=19, text='the adventure of the cardboard box by sir arthur conan doyle in choosing a few typical cases which illustrate the remarkable mental qualities of my friend sherlock holmes i have endeavoured as far as possible to select those which presented the minimum of sensationalism while offering a fair field for his talents it is however unfortunately impossible entirely to separate the sensational from the criminal and a chronicler is left in the dilemma that he must either sacrifice details which are essential to his statement and so give a false impression of the problem or he must use matter which chance and not choice has provided him with with this short preface i shall turn to my notes of what proved to be a strange though a peculiarly terrible chain of events it was a blazing hot day in august baker street was like an oven and the glare of the sunlight upon the yellow brickwork of the house across the road was painful to the eye it was hard to believe that these wer

In [5]:
d = spark.createDataFrame(df)

In [7]:
d.take(1)

[Row(book_id=0, text='illustration had she taken sides with either of them with a single movement the victory would have been decided in that favor the crystal stopper by maurice leblanc contents chapter i the arrests ii eight from nine leaves one iii the home life of alexis daubreco iv the chief of the enemies the vi the vii the profile of napoleon viii the tower ix in the dark xi the cross of lorraine xii the scaffold xiii the last battle illustrations had she taken sides with either of them with a single movement the victory would have been decided in that favor lupin took his servant by the shoulders and shook him said beaumont are you sure brought you the indomitable chief of our enemies have you a feeding bottle quiet be quiet she cried clutching him fiercely say lupin sprang to his feet he was prepared for every upshot except this we have to do is to stop the mischief and you understand the thing will be the sight which she beheld struck her with stupefaction daubrecq ran up to 

In [100]:
meta = pd.read_csv("data/primary_authors/metadata.csv")

In [102]:
meta.author.value_counts()

author
Doyle, Arthur Conan                  84
Chesterton, G. K. (Gilbert Keith)    61
Leblanc, Maurice                     20
Christie, Agatha                     12
Name: count, dtype: int64

In [37]:
t = torch.randn(4, 4)
t

tensor([[ 0.3928,  0.9512,  0.8813,  0.0199],
        [ 1.3431, -1.0503,  1.1941, -1.2154],
        [-2.0464,  1.4371,  1.6788,  0.0622],
        [-1.5129,  2.2014,  0.4617, -0.3892]])

In [39]:
torch.argmax(t, axis=1)

tensor([1, 0, 2, 1])

In [2]:
s = pd.Series([str(x % 8) for x in range(100)])

In [19]:
s.unique()

array(['0', '1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [3]:
df = pd.read_parquet("data/primary_authors/dataset.parquet")
df = book_train_test_split(df)

In [44]:
tfidf, we = extract_features("data/primary_authors/dataset.parquet", config_name="primary_authors")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=SparkTFIDF, master=local[*]) created by __init__ at /tmp/ipykernel_150470/882900860.py:6 

In [62]:
tbl = pd.read_parquet("data/primary_authors/tfidf/svm_preds_and_labels.parquet")
tbl

,pred,label
0,2,0
1,2,0
2,2,0
3,2,0
4,2,0
...,...,...
15208,2,3
15209,2,3
15210,2,3
15211,2,3


In [72]:
np.loadtxt("data/primary_authors/tfidf/gaussian/confusion_matrix.txt")

array([[8103.,    0.,    0.,    0.],
       [3966.,    0.,    0.,    0.],
       [2382.,    0.,    0.,    0.],
       [ 762.,    0.,    0.,    0.]])

In [42]:
d

{3258: 0, 3304: 1, 5036: 2, 10294: 3}

In [44]:
tbl['author_id'] = tbl.author_id.apply(lambda x: d[x])

In [54]:
tbl.features

0        [0.0113648325, 0.009577184, 0.0042619244, 0.00...
1        [0.013048511, 0.015733944, 0.0056825657, 0.006...
2        [0.012627591, 0.011629437, 0.00331483, 0.00673...
3        [0.010943913, 0.019154368, 0.007103207, 0.0077...
4        [0.012206672, 0.017786197, 0.004735471, 0.0077...
                               ...                        
19004    [0.011785752, 0.0143657755, 0.00994449, 0.0067...
19005    [0.011785752, 0.019154368, 0.00662966, 0.00399...
19006    [0.010522993, 0.017102113, 0.008050301, 0.0037...
19007    [0.01389035, 0.013681691, 0.0042619244, 0.0048...
19008    [0.012206672, 0.011629437, 0.008050301, 0.0058...
Name: features, Length: 19009, dtype: object

In [45]:
tbl.author_id

0        0
1        0
2        0
3        0
4        0
        ..
19004    3
19005    3
19006    3
19007    3
19008    3
Name: author_id, Length: 19009, dtype: int64

In [26]:
torch.Tensor(np.array(tbl[tbl.is_train].features.tolist()))

tensor([[0.0164, 0.0109, 0.0047,  ..., 0.0101, 0.0144, 0.0096],
        [0.0126, 0.0123, 0.0066,  ..., 0.0128, 0.0123, 0.0070],
        [0.0173, 0.0055, 0.0052,  ..., 0.0109, 0.0096, 0.0063],
        ...,
        [0.0072, 0.0130, 0.0085,  ..., 0.0116, 0.0089, 0.0085],
        [0.0114, 0.0130, 0.0085,  ..., 0.0114, 0.0089, 0.0074],
        [0.0168, 0.0116, 0.0066,  ..., 0.0118, 0.0103, 0.0077]])

In [88]:
t = pd.DataFrame(tfidf['features'].apply(lambda a: a['values']).tolist())
# t = pd.DataFrame(we['final_vector'].apply(lambda a: a['values']).tolist())
t = t.fillna(0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.009681,0.009577,0.010418,0.005471,0.015628,0.011365,0.013682,0.011839,0.006839,0.012207,0.009207,0.008523,0.006156,0.002578,0.008419,0.024310,0.006839,0.014731,0.014366,0.006261
1,0.013469,0.010261,0.013733,0.005471,0.013313,0.008997,0.018470,0.014206,0.009733,0.010102,0.008471,0.007576,0.004735,0.001473,0.006314,0.019680,0.009470,0.010101,0.018470,0.007366
2,0.010102,0.019154,0.009944,0.007365,0.017943,0.008997,0.010261,0.012786,0.006839,0.005893,0.006629,0.010733,0.004735,0.004420,0.005262,0.017364,0.006576,0.012626,0.015050,0.014364
3,0.014311,0.012314,0.008997,0.006313,0.014470,0.008524,0.021891,0.010892,0.008944,0.009681,0.007734,0.011364,0.010418,0.004051,0.007366,0.012155,0.006313,0.008838,0.015734,0.011049
4,0.009681,0.014366,0.009944,0.008207,0.016207,0.007577,0.021207,0.012312,0.008418,0.009260,0.005893,0.010733,0.006156,0.001841,0.007893,0.015628,0.007102,0.008838,0.019838,0.012154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19004,0.015574,0.017786,0.011365,0.006734,0.017943,0.007577,0.005473,0.009944,0.006576,0.007997,0.004051,0.009786,0.007103,0.007366,0.006840,0.021416,0.007366,0.012205,0.012314,0.011049
19005,0.013890,0.010261,0.006630,0.008838,0.020837,0.008050,0.009577,0.013733,0.008418,0.015995,0.005156,0.007576,0.008524,0.004420,0.012102,0.023731,0.004998,0.009049,0.008209,0.008839
19006,0.013890,0.015734,0.006156,0.007997,0.013313,0.011839,0.011629,0.011365,0.008418,0.011786,0.005893,0.009155,0.006156,0.005156,0.011050,0.026046,0.007102,0.012205,0.002736,0.006261
19007,0.013469,0.010945,0.004262,0.007786,0.019101,0.013733,0.013682,0.011365,0.009733,0.013049,0.006629,0.008523,0.005683,0.006629,0.006840,0.025468,0.006839,0.009470,0.007525,0.006629


In [89]:
len(t.iloc[0])

20

In [90]:
t_test = t[~df.is_train]
t_train = t[df.is_train]
y_test = df.author_id[~df.is_train].cat.codes
y_train = df.author_id[df.is_train].cat.codes


In [57]:
from sklearn.svm import SVC
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
import tensorflow as tf

2024-11-22 15:52:33.828412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732290753.845830   32075 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732290753.852497   32075 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 15:52:33.877344: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [62]:
class LSTM_Torch(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM_Torch, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=300, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, num_classes) #fully connected 1
        # self.fc = nn.Linear(128, 1) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        tag_space = self.fc_1(output.view(len(x), -1))
        scores = F.log_softmax(tag_space, dim=1)
        return scores
        
        # hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        # out = self.relu(hn)
        # out = self.fc_1(out) #first Dense
        # # out = self.relu(out) #relu
        # # out = self.fc(out) #Final Output
        # return out

In [91]:
t_tensor = torch.Tensor(t_train.to_numpy())
y_tensor = torch.Tensor(y_train.to_numpy())
t_final = torch.reshape(t_tensor, (t_tensor.shape[0],1,t_tensor.shape[1]))
dataset = TensorDataset(t_final, y_tensor)

t_test_tensor = torch.Tensor(t_test.to_numpy())
y_test_tensor = torch.Tensor(y_test.to_numpy())
t_test_final = torch.reshape(t_test_tensor, (t_test_tensor.shape[0],1,t_test_tensor.shape[1]))
test_dataset = TensorDataset(t_test_final, y_test_tensor)

In [92]:
t_tensor

tensor([[0.0118, 0.0130, 0.0114,  ..., 0.0097, 0.0150, 0.0088],
        [0.0105, 0.0103, 0.0095,  ..., 0.0124, 0.0103, 0.0110],
        [0.0143, 0.0082, 0.0114,  ..., 0.0105, 0.0144, 0.0129],
        ...,
        [0.0139, 0.0157, 0.0062,  ..., 0.0122, 0.0027, 0.0063],
        [0.0135, 0.0109, 0.0043,  ..., 0.0095, 0.0075, 0.0066],
        [0.0004, 0.0007, 0.0005,  ..., 0.0007,    nan,    nan]])

In [76]:


model = LSTM_Torch(4, t_tensor.shape[0], 2, 1, 4)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_fn = torch.nn.NLLLoss()
for i in range(0, 10):
    print(i)
    for data, labels in DataLoader(dataset, batch_size=16):
        labels = labels.type(torch.LongTensor)
    
        optimizer.zero_grad()
        outputs = model(data)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

all_preds = []
all_labels = []
with torch.no_grad():
    for data, labels in DataLoader(test_dataset, batch_size=16):
        data = data
        labels = labels.type(torch.LongTensor)

        preds = model(data)
        test_preds = torch.argmax(preds, axis=1)
        all_preds.extend(test_preds.numpy())
        all_labels.extend(labels.numpy())

0
1
2
3
4
5
6
7
8
9


In [79]:
t_tensor.shape(

torch.Size([3802, 300])

In [78]:
all_labels

[np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.in

In [28]:
arr = np.array(t[df.is_train])
arr.reshape((arr.shape[0],1,arr.shape[1]))

array([[[ 0.02870673,  0.09970236, -0.15648854, ...,  0.02137689,
          0.04841836,  0.07517256]],

       [[ 0.02870673,  0.09970236, -0.15648854, ...,  0.02137689,
          0.04841836,  0.07517256]],

       [[ 0.02870673,  0.09970236, -0.15648854, ...,  0.02137689,
          0.04841836,  0.07517256]],

       ...,

       [[ 0.03368383,  0.08731421, -0.13608781, ...,  0.03363849,
          0.02873958,  0.04364791]],

       [[ 0.03368383,  0.08731421, -0.13608781, ...,  0.03363849,
          0.02873958,  0.04364791]],

       [[ 0.03368383,  0.08731421, -0.13608781, ...,  0.03363849,
          0.02873958,  0.04364791]]])

In [33]:
df['author_id'].cat.codes

0        0
1        0
2        0
3        0
4        0
        ..
19004    3
19005    3
19006    3
19007    3
19008    3
Length: 19009, dtype: int8

In [7]:
df.shape

(19009, 3)

In [8]:
t = pd.read_parquet("data/primary_authors/document_embeddings.parquet")

In [11]:
t.shape

(19009, 3)

In [27]:
df2 = book_train_test_split(df)['is_train']
df2

/media/volume/team11data/524Project-Group11/proj2/dataset_handling.py:67: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count_df = df.groupby(['author_id', 'book_id']).count().reset_index()
/media/volume/team11data/524Project-Group11/proj2/dataset_handling.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sub_df = pd.concat([sub_df, book_row])


0         True
1         True
2         True
3         True
4         True
         ...  
19004    False
19005    False
19006    False
19007    False
19008    False
Name: is_train, Length: 19009, dtype: bool

In [50]:
tbl = pd.read_parquet("data/primary_authors/dataset.parquet")

In [51]:
tbl

,book_id,text,author_id
0,0,illustration had she taken sides with either o...,10294
1,0,plan and when i do a thing myself i am only go...,10294
2,0,avenue de ceinture and lupin and his two compa...,10294
3,0,of it but what there is is pucka nothing the m...,10294
4,0,to the house urged by a feeling of anxiety whi...,10294
...,...,...,...
19004,83,the last named to take over his flat the lette...,5036
19005,83,explained that he adopted the title of dentist...,5036
19006,83,sentenced to seven penal servitude on the evid...,5036
19007,83,own behalf he was advised by his counsel not t...,5036


In [48]:
ls = np.array([[x, x, x] for x in range(50)])
ls.reshape((ls.shape[0], 1, ls.shape[1]))

array([[[ 0,  0,  0]],

       [[ 1,  1,  1]],

       [[ 2,  2,  2]],

       [[ 3,  3,  3]],

       [[ 4,  4,  4]],

       [[ 5,  5,  5]],

       [[ 6,  6,  6]],

       [[ 7,  7,  7]],

       [[ 8,  8,  8]],

       [[ 9,  9,  9]],

       [[10, 10, 10]],

       [[11, 11, 11]],

       [[12, 12, 12]],

       [[13, 13, 13]],

       [[14, 14, 14]],

       [[15, 15, 15]],

       [[16, 16, 16]],

       [[17, 17, 17]],

       [[18, 18, 18]],

       [[19, 19, 19]],

       [[20, 20, 20]],

       [[21, 21, 21]],

       [[22, 22, 22]],

       [[23, 23, 23]],

       [[24, 24, 24]],

       [[25, 25, 25]],

       [[26, 26, 26]],

       [[27, 27, 27]],

       [[28, 28, 28]],

       [[29, 29, 29]],

       [[30, 30, 30]],

       [[31, 31, 31]],

       [[32, 32, 32]],

       [[33, 33, 33]],

       [[34, 34, 34]],

       [[35, 35, 35]],

       [[36, 36, 36]],

       [[37, 37, 37]],

       [[38, 38, 38]],

       [[39, 39, 39]],

       [[40, 40, 40]],

       [[41, 41,

In [33]:
tbl2[df2]

,book_id,final_vector,author_id
0,5,"{'type': 1, 'size': None, 'indices': None, 'va...",10294
1,0,"{'type': 1, 'size': None, 'indices': None, 'va...",10294
2,6,"{'type': 1, 'size': None, 'indices': None, 'va...",10294
3,16,"{'type': 1, 'size': None, 'indices': None, 'va...",10294
4,8,"{'type': 1, 'size': None, 'indices': None, 'va...",10294
...,...,...,...
18499,0,"{'type': 1, 'size': None, 'indices': None, 'va...",5036
18500,33,"{'type': 1, 'size': None, 'indices': None, 'va...",5036
18501,47,"{'type': 1, 'size': None, 'indices': None, 'va...",5036
18502,79,"{'type': 1, 'size': None, 'indices': None, 'va...",5036
